In [ ]:
import numpy as np
import pandas as pd
#from IPython.display import display

#import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

#import folium
#import folium.plugins
#import geopy
import geopandas
#import pygeocoder as geo

In [ ]:
bp17 = pd.read_csv('Building_Permits_in_2017.csv')

In [ ]:
bp17.columns

In [ ]:
pd.options.display.max_columns = 50

In [ ]:
bp17.head()

In [ ]:
x = pd.to_datetime(bp17['ISSUE_DATE'])
y = pd.to_datetime(bp17['LASTMODIFIEDDATE'])
bp17['TIMEDIF'] = (y-x).dt.days

In [ ]:
bp17.groupby('WARD')['TIMEDIF'].mean()

In [ ]:
pattern = r'[^0-9]'
bp17['SSLn'] = bp17['SSL'].str.replace(pattern, '')

In [ ]:
bp17['SSLn'].value_counts().head(10)

In [ ]:
bp17['Y'].value_counts().head(10)

In [ ]:
len(bp17)
len(bp17.groupby('SSLn').count())

In [ ]:
by_stype = bp17.groupby(['PERMIT_TYPE_NAME', 'PERMIT_SUBTYPE_NAME'])['OBJECTID'].count()
by_type = bp17.groupby('PERMIT_TYPE_NAME')['OBJECTID'].count()
by_type/len(bp17)*100

In [ ]:
sns.barplot(by_type.index, by_type)
plt.title('Permits filed by type, 2017')
plt.xlabel('number of permits filed')
plt.ylabel('category of permit')
plt.xticks(rotation=45);

In [ ]:
g = bp17.groupby('PERMIT_TYPE_NAME')['FEES_PAID'].median()

In [ ]:
sns.barplot(g.index, g)
plt.xticks(rotation=45)
plt.title('Median fee paid by permit type')
plt.ylabel('cost of permit ($)')
plt.xlabel('type of permit');

In [ ]:
bp17.groupby('APPLICATION_STATUS_NAME')['OBJECTID'].count()

In [ ]:
bp17['ISSUED'] = (bp17['APPLICATION_STATUS_NAME']=='PERMIT ISSUED')*1
bp17['DENIED'] = (bp17['APPLICATION_STATUS_NAME'].isin(['APPLICATION CANCELED', 'PERMIT CANCELED', 'PERMIT CANCELED REFUND', 'PERMIT REVOKED']))*1

In [ ]:
len(bp17[bp17['APPLICATION_STATUS_NAME']=='ISSUED'].groupby('WARD').count()['OBJECTID'])

In [ ]:
ward_count = bp17[bp17['APPLICATION_STATUS_NAME']=='PERMIT ISSUED'].groupby('WARD').count()['OBJECTID']/len(bp17[bp17['APPLICATION_STATUS_NAME']=='PERMIT ISSUED'])*100
pop_ward = np.array([76197, 79915, 77152, 75773, 74308, 76598, 71068, 70712])
white = np.array([36864, 57317, 64447, 18601, 12259, 38047, 1291, 2645])
black =  np.array([24794, 10079, 3860, 44459, 56489, 31842, 67471, 66131])
non_white = pop_ward - white
t = pd.DataFrame({'Permits Issued': ward_count,
                  'Population': pop_ward/sum(pop_ward)*100,
                  'Non-White Population': non_white/sum(non_white)*100,
                  'Ward': ward_count.index})
p = pd.melt(t.drop('Non-White Population', axis=1), id_vars='Ward')
q = pd.melt(t.drop('Population',axis=1), id_vars='Ward')

In [ ]:
sns.barplot(ward_count.index, ward_count)
plt.title('Permits issued by Ward')
plt.ylabel('% total');

In [ ]:
iss = bp17.groupby('WARD')['DENIED'].mean()*100
sns.barplot(iss.index, iss)
plt.title('Percentage of Permits denied');

In [ ]:
sns.barplot(x='Ward', y='value', hue='variable', data=p)
plt.title('Permits issued by Ward and Population')
plt.ylabel('% total')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

In [ ]:
sns.barplot(x='Ward', y='value', hue='variable', data=q)
plt.title('Permits issued by Ward and White Population')
plt.ylabel('% total')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

In [ ]:
bp17.groupby('WARD')['SSLn'].count().unique()

In [ ]:
x = np.array([1,2,3,4,5,6,7,8])
ar = [np.mean(bp17[bp17['WARD']==i].groupby('SSLn')['OBJECTID'].count()) for i in range(1,9)]
sns.barplot(x, ar)
plt.title('Average number of permits processed per business by ward (min=1)');

In [ ]:
by_ssl1 = bp17.groupby('SSLn').count()
by_ssl2 = bp17.groupby('SSLn').first()
by_ssl = pd.DataFrame({'SSLn': by_ssl1.index,
                       'Permit num': by_ssl1['OBJECTID'],
                       'Ward': by_ssl2['WARD']})

sns.boxplot(x='Ward', y='Permit num', data=by_ssl)

In [ ]:
sns.distplot(bp17.groupby('SSLn')['OBJECTID'].count(), bins=94)
plt.title('Frequency distribution - number of permits processed per business')
plt.xlabel('Number of permits filed')
plt.xlim((0,15));

In [ ]:
n  = bp17.groupby('SSLn')['OBJECTID'].count()
len(n[n>9])

# MAPPING

In [ ]:
dc_coord = (38.9108795, -77.0350123)
m = folium.Map(location=dc_coord, zoom_start=11.5)
m 

In [ ]:
counts = bp17.groupby(['﻿X', 'Y'])['OBJECTID'].count()


In [ ]:
by_ssl[by_ssl['Permit num']==2].head(10)

In [ ]:
bp17[bp17['SSLn']=='00162004']